## Project Overview,
A comprehensive tool for identifying security vulnerabilities in mobile application APIs.,

### Key Features,
- Insecure authentication mechanism detection\n,
- HTTPS encryption verification\n,
- Sensitive data exposure detection\n,
- Flexible and extensible testing framework

pip install requests colorama urllib3 jsonschema

In [1]:
import requests
import re
import json
import urllib3
from colorama import init, Fore, Style
import logging

In [1]:

# Initialize colorama for colored output
init(autoreset=True)

class MobileAPISecurityTester:
    def __init__(self, base_url, headers=None, auth_token=None):
        """
        Initialize the API Security Tester

        :param base_url: Base URL of the API
        :param headers: Optional custom headers
        :param auth_token: Optional authentication token
        """
        self.base_url = base_url
        self.headers = headers or {}
        self.auth_token = auth_token
        self.vulnerabilities = []
        self.logger = self._setup_logger()

    def _setup_logger(self):
        """Set up logging configuration"""
        logger = logging.getLogger('APISecurityTester')
        logger.setLevel(logging.INFO)
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s - %(levelname)s: %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        return logger

    def check_https(self, endpoint):
        """Check if the endpoint uses HTTPS"""
        if not endpoint.startswith('https://'):
            self.vulnerabilities.append({
                'type': 'HTTPS',
                'severity': 'HIGH',
                'description': f'Endpoint {endpoint} does not use HTTPS'
            })
            self.logger.warning(f'{Fore.RED}HTTPS Vulnerability: {endpoint} does not use secure HTTPS{Style.RESET_ALL}')

    def test_weak_authentication(self, endpoint, method='GET', payload=None):
        """Test authentication mechanisms"""
        try:
            # Test without authentication
            response_no_auth = requests.request(method, endpoint, data=payload)

            if response_no_auth.status_code < 401:
                self.vulnerabilities.append({
                    'type': 'WEAK_AUTH',
                    'severity': 'CRITICAL',
                    'description': 'Endpoint accessible without authentication'
                })
                self.logger.error(f'{Fore.RED}Authentication Vulnerability: Endpoint accessible without credentials{Style.RESET_ALL}')
        except Exception as e:
            self.logger.error(f'Error testing authentication: {e}')

    def detect_sensitive_data(self, response):
        """Detect potential sensitive data exposure"""
        sensitive_patterns = [
            r'(api_?key)\s*[=:]\s*[\'"]*([\\w-]+)',
            r'(password)\s*[=:]\s*[\'"]*([\\w-]+)',
            r'(secret)\s*[=:]\s*[\'"]*([\\w-]+)',
            r'\\b([\\w.-]+@[\\w.-]+\\.\\w+)\\b',  # Email detection
            r'\\b(\\d{3}-\\d{2}-\\d{4})\\b'  # SSN detection
        ]

        for pattern in sensitive_patterns:
            matches = re.findall(pattern, str(response.text), re.IGNORECASE)
            if matches:
                self.vulnerabilities.append({
                    'type': 'SENSITIVE_DATA',
                    'severity': 'HIGH',
                    'description': f'Sensitive data exposed: {matches}'
                })
                self.logger.warning(f'{Fore.YELLOW}Sensitive Data Exposure: {matches}{Style.RESET_ALL}')

    def generate_report(self):
        """Generate a comprehensive security vulnerability report"""
        report = {
            'total_vulnerabilities': len(self.vulnerabilities),
            'vulnerabilities': self.vulnerabilities
        }
        return json.dumps(report, indent=2)

    def run_comprehensive_test(self, endpoints):
        """Run comprehensive security tests on multiple endpoints"""
        for endpoint in endpoints:
            self.check_https(endpoint)
            self.test_weak_authentication(endpoint)

            try:
                response = requests.get(endpoint)
                self.detect_sensitive_data(response)
            except Exception as e:
                self.logger.error(f'Error testing endpoint {endpoint}: {e}')

def add_custom_vulnerability_check(tester, check_function):
    """Extend vulnerability detection with custom checks"""
    tester.custom_checks.append(check_function)

def main():
    # Sample API endpoints to test
    test_endpoints = [
        'http://vulnerable-api.example.com/users',
        'https://secure-api.example.com/data'
    ]

    # Initialize the security tester
    api_tester = MobileAPISecurityTester(base_url='https://example.com')

    # Run comprehensive tests
    api_tester.run_comprehensive_test(test_endpoints)

    # Generate and print the security report
    report = api_tester.generate_report()
    print(report)

if __name__ == '__main__':
    main()

NameError: name 'init' is not defined